# Modifications
1. Reduced Input Dimension
2. Different handling of xco2: merge the value in as an additional input in the fully connected layer (?)

In [ ]:
import imp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split

!module load cuda11.0/toolkit cuda11.0/blas cudnn8.0-cuda11.0

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

tf.config.list_physical_devices()

sys.path.insert(0, '../src')

from utils import df_to_xarray,read_xarray,plot_image,preprocess_image_reduced

/burg/glab/users/sk4973/venv/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [ ]:
# Reading Data
dir_name="../data/member_001"
chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2=read_xarray(dir_name)



In [ ]:
chl_images=preprocess_image_reduced(chl.Chl.data)
mld_images=preprocess_image_reduced(mld.MLD.data)
sss_images=preprocess_image_reduced(sss.SSS.data)
sst_images=preprocess_image_reduced(sst.SST.data)
xco2_images=preprocess_image_reduced(xco2_images)
pco2_images=preprocess_image_reduced(pco2.pCO2.data,True)

X = np.stack((chl_images, mld_images, sss_images, sst_images,xco2_images), axis = 1)


In [ ]:
X=X.reshape((421,180,360,5))
X.shape

In [ ]:
INPUT_SHAPE=X[0].shape
OUTPUT_SHAPE=pco2_images[0].shape

In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=4,activation='relu', padding="SAME")


base_model = keras.models.Sequential([
    DefaultConv2D(filters=64, input_shape=INPUT_SHAPE),
    DefaultConv2D(filters=64),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),

    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.Dropout(0.3),

    keras.layers.UpSampling2D(size=2),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    
    DefaultConv2D(filters=2),
    DefaultConv2D(filters=2),
    DefaultConv2D(filters=1,kernel_size=1,activation='sigmoid'),
    keras.layers.Reshape(OUTPUT_SHAPE)
   
])

base_model.summary()

In [ ]:
base_model.compile(loss="mean_squared_error", optimizer="nadam", metrics=["mean_squared_error"])

In [ ]:
model_path="../models/base_model/base_model.h5"
early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

history = base_model.fit(X,pco2_images, epochs=100, validation_data=(X,pco2_images),workers=-1,batch_size=32,callbacks=callbacks)

